Reference

- https://github.com/adapter-hub/adapters
- https://docs.adapterhub.ml/loading.html
- https://docs.adapterhub.ml/adapter_composition.html


-https://huggingface.co/docs/transformers/peft

In [2]:
!sudo -H pip install -Uq transformers

In [2]:
import os
import torch
from datasets import load_dataset
from datasets import Dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
# from peft import LoraConfig, PeftModel
# from trl import SFTTrainer
import pandas as pd
import torch

In [ ]:
################################################################################
# bitsandbytes parameters
################################################################################

# Activate 4-bit precision base model loading
use_4bit = True

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = True


# Load tokenizer and model with QLoRA configuration
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)


model_name = "mistralai/Mistral-7B-Instruct-v0.1"

# Load base model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
#     quantization_config=bnb_config,
    device_map="auto",
#     load_in_8bit=False,
    torch_dtype=torch.float16
)
model.config.use_cache = False
model.config.pretraining_tp = 1

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [6]:
!sudo -H pip install peft

     |████████████████████████████████| 184kB 4.8MB/s eta 0:00:01


In [1]:
from peft import PeftModel, PeftConfig

/packages/Transformers_CPU/8d52bc28-4231-418c-8b06-7a9c433a1b7a/3.8/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


/packages/Transformers_CPU/8d52bc28-4231-418c-8b06-7a9c433a1b7a/3.8/bitsandbytes/libbitsandbytes_cpu.so: undefined symbol: cadam32bit_grad_fp32


TypeError: 'type' object is not subscriptable

In [8]:
adapter1 = "/data/mistral/query-to-mql/exp-10/nov-20/checkpoint-8000"
adapter2 = "/data/mistral/query-to-mql/exp-10/nov-20/checkpoint-4000"

In [9]:
# using load_adapter

model.load_adapter(adapter1, adapter_name = 'adapter1')


ValueError: PEFT is not installed. Please install it with `pip install peft`

In [49]:
model.load_adapter(adapter2, adapter_name = 'adapter2')

In [50]:
%%time

model.set_adapter('adapter1')

CPU times: user 2 ms, sys: 2.63 ms, total: 4.64 ms
Wall time: 12.6 ms


In [18]:
%%time
model_to_merge = PeftModel.from_pretrained(model,adapter1)

CPU times: user 1.05 s, sys: 0 ns, total: 1.05 s
Wall time: 376 ms


In [33]:
%%time
model_to_merge2 = PeftModel.from_pretrained(model,adapter2)
model_to_merge2.to('cuda')

CPU times: user 1.1 s, sys: 4.36 ms, total: 1.11 s
Wall time: 403 ms


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): MistralForCausalLM(
      (model): MistralModel(
        (embed_tokens): Embedding(32000, 4096)
        (layers): ModuleList(
          (0-31): 32 x MistralDecoderLayer(
            (self_attn): MistralAttention(
              (q_proj): Linear4bit(
                in_features=4096, out_features=4096, bias=False
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=64, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=64, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
              )
              (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
              (v_proj)

In [53]:
def predict_template_query_v1(user_query):
    inp = query_template_v1.format(context=context,
                                   user_query=user_query,
                                  date_input=date_input)
    _inputs = tokenizer.encode(inp, return_tensors="pt").to('cuda')
    outputs = model.generate(input_ids=_inputs, max_length= 1200, pad_token_id=tokenizer.eos_token_id)
    output = tokenizer.decode(outputs[0])
    output_new = output.split('[MQL]\n')[1]
    return output_new.split('\n[/MQL]')[0], output

In [41]:
import time

In [54]:
start = time.time()
user_query = 'what is purchase across segments'
print('user query: ', user_query)
print('-'*100)
output, raw = predict_template_query_v1(user_query=user_query)
print("time taken : ", time.time()-start)

user query:  what is purchase across segments
----------------------------------------------------------------------------------------------------
time taken :  482.13821148872375


In [38]:
output

"{'DIMENSION': {'segments': [{'ENTITY': 'Segment', 'RANK': [{'RANK ADJECTIVE': '', 'RANK VALUE': ''}]}]}, 'MEASURE': {'purchase': [{'ENTITY': 'Purchase Vol', 'MEASURE CONSTRAINT': [{'COMPARISON VALUE': '', 'COMPARSION OPERATOR': ''}]}]}}"

In [22]:
context = """{
    "MEASURE": [{"ENTITY": "Discount", "other names": ["discount", "discount rate", "discount value", "deduction"]},
                {"ENTITY": "Purchase Vol", "other names": ["purchase", "purchase value", "purchase model"]},
                {"ENTITY": "Quantity", "other names": ["quantity", "volume"]},
                {"ENTITY": "Sales", "other names": ["sales", "sale"]}],
    "DIMENSION": [{"ENTITY": "Sub-Category", "other names": ["sub-category", "sub category", "categories", "section"]},
                  {"ENTITY": "Segment", "other names": ["segment", "segments", "units", "divisions"]},
                  {"ENTITY": "Parts", "other names": ["parts", "part", "section", "divisions"]},
                  {"ENTITY": "Country", "other names": ["country", "countries"]}],
    "FILTER": [{"ENTITY": "Consumer", "other names": ["consumers", "consumer"], "parent": "Segment"},
               {"ENTITY": "Phone", "other names": ["phone", "phones", "mobile phones"], "parent": "Sub-Category"},
               {"ENTITY": "Binder", "other names": ["binders", "binder"], "parent": "Sub-Category"},
               {"ENTITY": "Corporate", "other names": ["corporates", "corporate"], "parent": "Segment"},
               {"ENTITY": "India", "other names": ["india"], "parent": "Country"},
               {"ENTITY": "Dubai", "other names": ["dubai"], "parent": "Country"}],
    "DERIVED MEASURE": [{"ENTITY": "Ratio",
             "other names": ["ratio", "share", "contribution", "percentage", "proportion", "contributing"]},
            {"ENTITY": "Why", "other names": ["why", "cause of", "reason for", "diagnose"]},
            {"ENTITY": "contribution_to_growth", "other names": ["contribution to growth", "growth", "grown"]},
            {"ENTITY": "kda_transactional", "other names": ["kda", "key drivers", "key driver", "drivers", "driver"]},
            {"ENTITY": "Growth Rate", "other names": ["growth rate", "growth", "grown"]},
            {"ENTITY": "correlation",
             "other names": ["associate", "associated", "association", "associations", "correlate", "correlated",
                             "correlation", "correlations", "relate", "related", "relation", "relations",
                             "relationship",
                             "relationships"]}
            ],
    "DATE VARIABLE": [{"ENTITY": "Order Date", "other names": ["order date", "date", "trend", "time", "when", "mom", "yoy"]}]
    }"""

date_input = {
    "start_date": "01/01/2020",
    "end_date": "15/09/2023"
}

In [23]:
query_template_v1 = """Given the context : {context} and date reference: {date_input}, the query: {user_query}, is converted into below shown structured output.
[MQL]
"""

In [ ]:
# model = AutoAdapterModel.from_pretrained("roberta-base")


model.load_adapter("AdapterHub/roberta-base-pf-imdb", source="hf", set_active=True)

print(model(**tokenizer("This works great!", return_tensors="pt")).logits)